Importing Relevant Libraries

In [1]:
import cv2
import dlib
import os
import csv
import numpy as np
from datetime import datetime
import urllib.request
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model, save_model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D

Use Pre Trained Face Detector to Crop Images 

In [2]:
def resize_images_with_face_focus(folder_path, output_folder, target_size=(300, 300)):
    # Load face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over the images in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            output_path = os.path.join(output_folder, filename)

            # Read the image
            image = cv2.imread(image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Detect faces in the image
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            if len(faces) > 0:
                # Select the largest face
                (x, y, w, h) = max(faces, key=lambda rect: rect[2] * rect[3])

                # Calculate the center coordinates of the face
                center_x = x + w // 2
                center_y = y + h // 2

                # Calculate the size of the square region around the face
                face_size = max(w, h)

                # Calculate the region of interest (ROI) coordinates
                roi_x = center_x - face_size // 2
                roi_y = center_y - face_size // 2

                # Extract the region of interest (ROI)
                roi = image[roi_y:roi_y + face_size, roi_x:roi_x + face_size]

                # Resize the ROI to the target size
                resized_roi = cv2.resize(roi, target_size)

                # Save the resized image
                cv2.imwrite(output_path, resized_roi)

                print(f"Processed image: {filename}")

            else:
                print(f"No faces found in the image: {filename}")

# Example usage
folder_path = 'Old'
output_folder = 'New'
resize_images_with_face_focus(folder_path, output_folder)


Processed image: 86717448_2419567865022562_3625521906192482304_n.jpg


Create Face Embeddings Tenserflow Model

In [6]:
# Path to the dataset
dataset_path = "Images"

def load_dataset(dataset_path):
    data = []
    labels = []
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            image = load_img(image_path, target_size=(64, 64))
            image = img_to_array(image)
            data.append(image)
            labels.append(label)

    data = np.array(data, dtype="float32")
    labels = np.array(labels)
    return data, labels

data, labels = load_dataset(dataset_path)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize the input data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Define the embedding model
embedding_dim = 2048
embedding_model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(embedding_dim, activation='relu'),
])

data, labels = load_dataset(dataset_path)

# Normalize the input data
data = data / 255.0

# Load the pre-trained model for calculating embeddings
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(64, 64, 3),
    pooling='avg'
)

# Calculate embeddings for all the images
embeddings = pretrained_model.predict(data)

# Split the data into training and validation sets
x_train2, x_val, y_train2, y_val = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Compile the embedding model
embedding_model.compile(optimizer='adam', loss='mse')

# Train the embedding model
embedding_model.fit(x_train, x_train2, epochs=30, validation_data=(x_test, x_val))

# Save the embedding model
embedding_model.save('face_embedding.h5')

5/5 [==============================] - 3s 332ms/step
Epoch 1/30
4/4 [==============================] - 3s 574ms/step - loss: 0.1878 - val_loss: 0.1061
Epoch 2/30
4/4 [==============================] - 2s 537ms/step - loss: 0.0640 - val_loss: 0.0301
Epoch 3/30
4/4 [==============================] - 2s 517ms/step - loss: 0.0302 - val_loss: 0.0264
Epoch 4/30
4/4 [==============================] - 2s 538ms/step - loss: 0.0286 - val_loss: 0.0257
Epoch 5/30
4/4 [==============================] - 2s 476ms/step - loss: 0.0226 - val_loss: 0.0194
Epoch 6/30
4/4 [==============================] - 2s 522ms/step - loss: 0.0204 - val_loss: 0.0183
Epoch 7/30
4/4 [==============================] - 3s 699ms/step - loss: 0.0198 - val_loss: 0.0182
Epoch 8/30
4/4 [==============================] - 2s 566ms/step - loss: 0.0187 - val_loss: 0.0169
Epoch 9/30
4/4 [==============================] - 2s 493ms/step - loss: 0.0182 - val_loss: 0.0175
Epoch 10/30
4/4 [==============================] - 2s 510ms/step 

Download Pre Trained Detection and Recognition Models

In [ ]:
# Download the shape predictor model if not already downloaded
shape_predictor_file = 'shape_predictor_68_face_landmarks.dat'
if not os.path.exists(shape_predictor_file):
    print("Downloading shape predictor model...")
    urllib.request.urlretrieve("http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2",
                               "shape_predictor_68_face_landmarks.dat.bz2")
    print("Extracting shape predictor model...")
    os.system("bzip2 -d shape_predictor_68_face_landmarks.dat.bz2")

Import Face Detector and Recognizer

In [7]:
# Set up face detector and face recognition models
detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# Load the TensorFlow face recognition model
model_file = 'face_embedding.h5'
face_recognizer = tf.keras.models.load_model(model_file)

Loading Training Data

In [8]:
# Path to the folder containing labeled images
data_folder = 'Images'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Path to the folder containing labeled images
data_folder = 'drive/MyDrive/Images'

In [9]:
# Load the labeled images and their corresponding names
labeled_images = []
labels = []
for label_name in os.listdir(data_folder):
    label_folder = os.path.join(data_folder, label_name)
    for image_name in os.listdir(label_folder):
        image_path = os.path.join(label_folder, image_name)
        labeled_images.append(cv2.imread(image_path))
        labels.append(label_name)

Calculating Embeddings for the Training Data

In [10]:
# Calculate face embeddings for the labeled images
embeddings = []
for image in labeled_images:
    faces = detector(image)
    for face in faces:
        shape = shape_predictor(image, face)
        (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
        face_region = image[y:y+h, x:x+w]  # Extract face region
        face_region = cv2.resize(face_region, (64, 64))  # Resize to match model input size
        face_region = np.expand_dims(face_region, axis=0)  # Add batch dimension
        embedding = face_recognizer.predict(face_region)[0]
        embeddings.append(embedding)

1/1 [==============================] - 0s 75ms/step


Creating/Accessing The Attendance CSV

In [13]:
# Set up attendance CSV file
csv_file = 'attendance.csv'
if not os.path.exists(csv_file):
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Attendance', 'Time'])  # Updated header row
        unique_labels = list(set(labels))  # Get unique labels
        for label_name in unique_labels:
            writer.writerow([label_name, 'Absent', ''])  # Write unique label names initially with "Absent" status

In [ ]:
from google.colab import drive
import csv
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set up attendance CSV file in Google Drive
csv_file = '/content/drive/MyDrive/attendance.csv'
if not os.path.exists(csv_file):
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Timestamp'])
        print("Attendance CSV file created.")
else:
    print("Attendance CSV file already exists.")

Running Detection using Device Camera

In [14]:
# Initialize video capture
cap = cv2.VideoCapture(0)

attendance_data = {}  # Store attendance data
last_label = None  # Store the last predicted label
stable_count = 0  # Count the number of stable predictions

# Load existing attendance data from CSV
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  # Get header row
    for row in reader:
        label = row[0]
        attendance = row[1]
        attendance_data[label] = attendance

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)

    # Iterate over detected faces
    for face in faces:
        # Determine face landmarks for alignment
        shape = shape_predictor(frame, face)
        (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
        face_region = frame[y:y+h, x:x+w]  # Extract face region
        face_region = cv2.resize(face_region, (64, 64))  # Resize to match model input size
        face_region = np.expand_dims(face_region, axis=0)  # Add batch dimension

        # Calculate face embedding for recognition
        embedding = face_recognizer.predict(face_region)[0]

        # Compare face embedding with labeled embeddings
        distances = np.linalg.norm(embeddings - np.expand_dims(embedding, axis=0), axis=1)
        min_distance_index = np.argmin(distances)
        min_distance = distances[min_distance_index]

        # Threshold for face recognition
        if min_distance < 150:
            label = labels[min_distance_index]

            # Check if the label exists in the attendance data
            if label in attendance_data:
                if last_label == label:
                    stable_count += 1
                    if stable_count == 10:
                        if attendance_data[label] == 'Present':
                            label = f'{label} (Already Marked)'
                            cv2.putText(frame, 'Marked', (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        else:
                            # Update attendance data to mark as Present
                            attendance_data[label] = 'Present'
                            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                            attendance_data[label] = 'Present'
                else:
                    last_label = label
                    stable_count = 0

            # Display recognized label on the frame
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Attendance System', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Update attendance
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)  # Write the header row
    for label, attendance in attendance_data.items():
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        if attendance == 'Present':
            writer.writerow([label, attendance, timestamp])  # Write the updated attendance data
        else:
            writer.writerow([label, attendance, ''])  # Write the absent attendance data

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 45ms/step
